In [51]:
from aocd import get_data

she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```

The actual word search will be full of letters instead. For example:

In [24]:
example_ws = """
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
"""

In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with `.`:

```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```

In [11]:
import numpy as np
import re

Get instances reading forward horizontally

In [33]:
rows = np.array(example_ws.strip().splitlines()); rows

array(['MMMSXXMASM', 'MSAMXMSMSA', 'AMXSXMAAMM', 'MSAMASMSMX',
       'XMASAMXAMM', 'XXAMMXXAMA', 'SMSMSASXSS', 'SAXAMASAAA',
       'MAMMMXMMMM', 'MXMXAXMASX'], dtype='<U10')

In [34]:
vectorized_extract_fwd = np.vectorize(lambda x: len(re.findall(r'XMAS', x)))
vectorized_extract_bwd = np.vectorize(lambda x: len(re.findall(r'SAMX', x)))

In [36]:
instances = 0
instances += np.sum(vectorized_extract_fwd(rows))
instances += np.sum(vectorized_extract_bwd(rows))

In [37]:
instances

np.int64(5)

Now a little bit of manipilation to get cols and diagonals

In [39]:
chars = np.array([list(line) for line in example_ws.strip().splitlines()])
cols = np.array([''.join(col) for col in chars.T]); cols

['MMAMXXSSMM',
 'MSMSMXMAAX',
 'MAXAAASXMM',
 'SMSMSMMAMX',
 'XXXAAMSMMA',
 'XMMSMXAAXX',
 'MSAMXXSSMM',
 'AMASAAXAMA',
 'SSMMMMSAMS',
 'MAMXMASAMX']

In [40]:
instances += np.sum(vectorized_extract_fwd(cols))
instances += np.sum(vectorized_extract_bwd(cols))

In [41]:
instances

np.int64(8)

In [53]:
def extract_diagonals(array):
    diagonals = []

    # main diagonals (bottom-left to top-right)
    for offset in range(-array.shape[0] + 1, array.shape[1]):
        diagonals.append(''.join(array.diagonal(offset)))

    # anti-diagonals (bottom-right to top-left)
    flipped_array = np.fliplr(array)
    for offset in range(-flipped_array.shape[0] + 1, flipped_array.shape[1]):
        diagonals.append(''.join(flipped_array.diagonal(offset)))

    return diagonals


diagonal_strings = extract_diagonals(chars)

# Can't do vectorized here because the diags vary in length
for diag in diagonal_strings:
    instances += len(re.findall(r'XMAS', diag))
    instances += len(re.findall(r'SAMX', diag))

In [50]:
instances

np.int64(18)

Now apply to our real data:

In [52]:
data = get_data(day=4, year=2024)

rows = np.array(data.strip().splitlines())
chars = np.array([list(line) for line in data.strip().splitlines()])
cols = np.array([''.join(col) for col in chars.T])
diagonal_strings = extract_diagonals(chars)

instances = 0
instances += np.sum(vectorized_extract_fwd(rows))
instances += np.sum(vectorized_extract_bwd(rows))
instances += np.sum(vectorized_extract_fwd(cols))
instances += np.sum(vectorized_extract_bwd(cols))
for diag in diagonal_strings:
    instances += len(re.findall(r'XMAS', diag))
    instances += len(re.findall(r'SAMX', diag))

instances

np.int64(2336)

## Part 2

We can search for instances of `A` and then check the diagonal chars for `M` and `S`.

Find all instances of A but ignore top row, bottom row, and left-most and right-most columns

In [83]:
a_chars = chars[1:-1, 1:-1] == 'A'

In [84]:
a_chars

array([[False,  True, False, ..., False, False, False],
       [False, False, False, ..., False,  True,  True],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [ True,  True, False, ...,  True, False, False]])

In [85]:
a_chars = np.pad(a_chars, ((1, 1), (1, 1)), constant_values=False)

In [86]:
a_chars.shape

(140, 140)

Now we know the co-ords of each `A` char that might be part of the solution, we can perform checks on whether their relatives chars match the `MAS` diagonal pattern:

In [88]:
coordinates = np.argwhere(a_chars)

In [89]:
coordinates

array([[  1,   2],
       [  1,   5],
       [  1,   7],
       ...,
       [138, 123],
       [138, 128],
       [138, 136]])

In [100]:
def extract_x_pattern(array, center_row, center_col):
    # rel positions for the 'X' pattern
    x_offsets = [(-1, -1), (-1, 1), (0, 0), (1, -1), (1, 1)]
    result = []
    
    for dr, dc in x_offsets:
        row, col = center_row + dr, center_col + dc
        result.append(array[row, col])
    
    return result

def check_x_pattern_for_words(array, coords, words):
    matches = []
    
    for center_row, center_col in coords:
        x_pattern = extract_x_pattern(array, center_row, center_col)
        # convert the extracted pattern into diagonals as strings
        diagonals = [
            "".join([x_pattern[i] for i in [0, 2, 4] if x_pattern[i] is not None]),  # top-left to bottom-right
            "".join([x_pattern[i] for i in [1, 2, 3] if x_pattern[i] is not None])   # top-right to bottom-left
        ]
        if all(diag in words for diag in diagonals):
            matches.append((center_row, center_col))
    
    return matches

words_to_check = ["MAS", "SAM"]
matches = check_x_pattern_for_words(chars, coordinates, words_to_check)


In [101]:
len(matches)

1831